# 04. 주별 분석 (State Analysis)

Critical Ratios를 사용하여 주별 성격 특성 차이를 분석합니다.

## 학습 목표
- Critical Ratio (CR) 개념 이해
- 주별 성격 차이 해석
- 유의미한 특징 식별 (|CR| > 3.0)

In [ ]:
# 필요한 라이브러리 설치
%pip install pandas numpy matplotlib seaborn -q

In [ ]:
# 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# 상위 폴더로 이동
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')
print(f'작업 폴더: {os.getcwd()}')

# 그래프 스타일
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.dpi'] = 100

## 1. 데이터 로드

In [ ]:
# 전처리된 점수 데이터 로드
scores = pd.read_csv('data/processed/sapa_scores.csv')
print(f'전체 데이터: {len(scores):,}명')
print(f'컬럼: {list(scores.columns)}')

In [ ]:
# State 분포 확인
state_counts = scores['state'].value_counts()
print('=== State별 표본 수 ===')
print(state_counts)

## 2. 분석 대상 선정

- "other" 제외 (국제 참가자 등)
- 충분한 표본 수가 있는 주만 분석 (N ≥ 100)

In [ ]:
# "other" 제외 및 충분한 표본 수 확인
valid_states = state_counts[(state_counts.index != 'other') & (state_counts >= 100)].index.tolist()

print(f'분석 대상 주: {len(valid_states)}개')
print(valid_states)

In [ ]:
# 분석용 데이터 필터링
df_analysis = scores[scores['state'].isin(valid_states)].copy()
print(f'분석 대상 인원: {len(df_analysis):,}명')

## 3. Critical Ratios 계산

**공식**: CR = (State Mean - Grand Mean) / SE

**기준**: |CR| > 3.0 → 유의미 (p < .003)

In [ ]:
# 7개 주요 척도 (실제 컬럼명!)
scales = ['NEO_O', 'NEO_C', 'NEO_E', 'NEO_A', 'NEO_N', 'Ideology', 'Honesty_Humility']
scale_labels = ['O', 'C', 'E', 'A', 'N', 'Ideology', 'H-H']

# Grand Mean 계산 (전체 분석 대상의 평균)
grand_means = df_analysis[scales].mean()
print('=== Grand Means ===')
for scale, label in zip(scales, scale_labels):
    print(f'{label}: {grand_means[scale]:.3f}')

In [ ]:
# State별 CR 계산
results = []

for state in valid_states:
    state_data = df_analysis[df_analysis['state'] == state]
    
    for scale, label in zip(scales, scale_labels):
        data = state_data[scale].dropna()
        n = len(data)
        
        if n > 0:
            mean = data.mean()
            se = data.std() / np.sqrt(n)
            cr = (mean - grand_means[scale]) / se if se > 0 else 0
            
            results.append({
                'state': state,
                'scale': label,
                'n': n,
                'mean': mean,
                'se': se,
                'cr': cr,
                'significant': abs(cr) > 3.0
            })

df_cr = pd.DataFrame(results)
print(f'CR 계산 완료: {len(df_cr)}개 (State × Scale)')

In [ ]:
# State별 표본 수 요약
state_n = df_cr.groupby('state')['n'].first().sort_values(ascending=False)
print('=== 분석 대상 State별 표본 수 ===')
for state, n in state_n.items():
    print(f'{state}: {n:,}명')
print(f'\n총 인원: {state_n.sum():,}명')

## 4. 유의미한 특징 식별

In [ ]:
# 유의미한 CR 추출 (|CR| > 3.0)
df_significant = df_cr[df_cr['significant']].copy()
df_significant['direction'] = df_significant['cr'].apply(lambda x: '높음' if x > 0 else '낮음')

print(f'=== 유의미한 특징 (|CR| > 3.0): {len(df_significant)}개 ===')
for _, row in df_significant.sort_values('cr', key=abs, ascending=False).iterrows():
    print(f"{row['state']} - {row['scale']}: CR = {row['cr']:.2f} ({row['direction']})")

## 5. 히트맵 시각화

In [ ]:
# CR 히트맵용 피벗 테이블
cr_pivot = df_cr.pivot(index='state', columns='scale', values='cr')

# 척도 순서 정렬
cr_pivot = cr_pivot[scale_labels]

# 히트맵
fig, ax = plt.subplots(figsize=(12, 8))

sns.heatmap(cr_pivot, 
            annot=True, 
            fmt='.2f', 
            cmap='RdBu_r', 
            center=0,
            vmin=-5, 
            vmax=5,
            linewidths=0.5,
            ax=ax,
            cbar_kws={'label': 'Critical Ratio'})

# 유의미한 셀 강조 (|CR| > 3.0)
for i, state in enumerate(cr_pivot.index):
    for j, scale in enumerate(cr_pivot.columns):
        cr_val = cr_pivot.loc[state, scale]
        if abs(cr_val) > 3.0:
            ax.add_patch(plt.Rectangle((j, i), 1, 1, fill=False, edgecolor='black', linewidth=2))

ax.set_title('State-Level Critical Ratios of Personality Scales\n(|CR| > 3.0 highlighted)', 
             fontsize=14, fontweight='bold')
ax.set_xlabel('Personality Scale', fontsize=12)
ax.set_ylabel('State', fontsize=12)

plt.tight_layout()

# 저장
os.makedirs('reports/figures', exist_ok=True)
plt.savefig('reports/figures/state_critical_ratios_heatmap.png', dpi=150, bbox_inches='tight')
print('저장: reports/figures/state_critical_ratios_heatmap.png')
plt.show()

## 6. 최종 요약

In [ ]:
print('=== State Analysis 완료 요약 ===')
print(f'\n[분석 대상]')
print(f'  - 주(State): {len(valid_states)}개')
print(f'  - 총 인원: {len(df_analysis):,}명')

print(f'\n[유의미한 특징 (|CR| > 3.0)]')
print(f'  - 총 {len(df_significant)}개')

if len(df_significant) > 0:
    print('\n  주요 발견:')
    for _, row in df_significant.sort_values('cr', key=abs, ascending=False).head(10).iterrows():
        direction = '↑' if row['cr'] > 0 else '↓'
        print(f"    {row['state']} - {row['scale']}: CR = {row['cr']:.2f} {direction}")

print(f'\n[생성된 그림]')
print('  - reports/figures/state_critical_ratios_heatmap.png')